In [ ]:
import requests
import json
import os
from dotenv import load_dotenv

## Config Meta App + Instagram Variables

In [ ]:
load_dotenv()

ig_user_id = os.getenv("IG_USER_ID")
app_id = os.getenv("APP_ID")
app_secret = os.getenv("APP_SECRET")
user_access_token = os.getenv("USER_ACCESS_TOKEN")

In [ ]:
https://graph.facebook.com/v19.0/oauth/access_token
?grant_type=fb_exchange_token
&client_id={app_id}
&client_secret={app_secret}
&fb_exchange_token={user_access_token}

## Get Long Access Token

In [ ]:
url = f"https://graph.facebook.com/v19.0/oauth/access_token?grant_type=fb_exchange_token&client_id={app_id}&client_secret={app_secret}&fb_exchange_token={user_access_token}"
response = requests.get(url)
#print(response.content)
long_access_token = response.json()["access_token"]
long_access_token

## Test API

In [ ]:
"https://graph.facebook.com/v23.0/17841405309211844 \
  ?fields=business_discovery.username(bluebottle){followers_count,media_count} \
  &access_token=<YOUR_APP_USERS_INSTAGRAM_USER_ACCESS_TOKEN>"

In [ ]:
# Get account details
username="jestestbusiness"
required_param = "{followers_count,media_count}"
url = f"https://graph.facebook.com/v19.0/{ig_user_id}?fields=business_discovery.username({username}){required_param}&access_token={long_access_token}"
response = requests.get(url)
metadata= response.json()
followers_count = metadata["business_discovery"]["followers_count"]
media_count = metadata["business_discovery"]["media_count"]
print(json.dumps(metadata, indent=4))

### Check User Page (Does not work on Private Accounts)

In [ ]:
ig_username = "photovogue"
required_param = "{name,website,biography,followers_count,follows_count,media_count,profile_picture_url,media.limit(5){thumbnail_url,media_type,media_product_type,timestamp,like_count,comments_count,media_url,permalink}}"

In [ ]:
url = f"https://graph.facebook.com/v19.0/{ig_user_id}?fields=business_discovery.username({ig_username}){required_param}&access_token={long_access_token}"
response = requests.get(url)
metadata = response.json()
print(json.dumps(metadata, indent=4))

### Download a Post and Thumbnail

In [ ]:
first_post_link = metadata["business_discovery"]["media"]["data"][0]["media_url"]
print(f"Media URL : {first_post_link} \n")
first_post_thumbnail_link = metadata["business_discovery"]["media"]["data"][0]["thumbnail_url"]
print(f"Media Thumbnail URL : {first_post_thumbnail_link}")

In [ ]:
os.makedirs("./temp", exist_ok=True)

response = requests.get(first_post_link)
with open ("./temp/first_post.mp4", "wb") as f:
    f.write(response.content)

In [ ]:
response = requests.get(first_post_thumbnail_link)
with open ("./temp/first_post_thumbnail_link.jpg", "wb") as f:
    f.write(response.content)

### Pagination

In [ ]:
{media.limit(1).after(your_after_code){your_fields_by_comma_separation}}

In [ ]:
after_code = metadata["business_discovery"]["media"]["paging"]["cursors"]["after"]
after_code

In [ ]:
required_param = "{media.after(" + after_code + "){thumbnail_url,media_type,media_product_type,timestamp,like_count,comments_count,media_url,permalink}}"
url = f"https://graph.facebook.com/v19.0/{ig_user_id}?fields=business_discovery.username({ig_username}){required_param}&access_token={long_access_token}"
response = requests.get(url)
metadata = response.json()
print(json.dumps(metadata, indent=4))